# Zero to Hero

## 1.15 Montecarlo Estimation, Arbol
## En una funcion

El objetivo es implementar con un for loop la estimacion montecarlo, que promedia las ganancias

In [1]:
library("data.table") # cargo la libreria  data.table
library("rpart") # cargo la libreria  rpart
#library("rpart.plot")
library("caret")

#options(repr.plot.width = 20, repr.plot.height = 10)
#setwd("~/buckets/b1/")
dataset <- fread("D:\\MAESTRIA AUSTRAL DATA SCIENCE\\MATERIAS\\2doSem\\Laboratorio 1\\datasets\\dataset_pequeno.csv")
head(dataset, 5)

Loading required package: lattice
Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang


numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
29183733,202107,1,0,0,46,99,2016.19,32146.69,1099.10,...,3,0.00,-27151.97,0,2995,10011.78,6,0,2498.49,CONTINUA
29184468,202107,1,0,0,42,284,1633.70,13781.99,602.61,...,10,0.00,-19268.21,0,5722,15138.18,9,0,3178.83,CONTINUA
29185245,202107,1,0,0,55,23,3759.73,24296.95,2829.38,...,3,0.00,-8211.00,0,758,58080.27,10,0,26240.01,CONTINUA
29186441,202107,1,0,0,62,296,2693.26,75390.56,2493.68,...,24,41585.05,-42723.62,0,8097,26541.92,8,0,1782.96,CONTINUA
29186475,202107,1,0,0,66,326,4567.63,56305.98,3690.64,...,3,0.00,-51675.11,0,2139,50304.17,16,0,3577.65,CONTINUA


La funcion que definimos en el notebook anterior

In [2]:
GananciaArbol <- function(semilla, data, x, train = 0.70) {
  # establezco la semilla
  set.seed(semilla)
  train_rows <- createDataPartition(dataset$clase_ternaria, p = 0.70, list = FALSE)

  modelo <- rpart::rpart(
    formula = "clase_ternaria ~ .",
    data = data[train_rows], # los datos de training
    xval = 0,
    control = x
  )

  # Aplico el modelo a los datos de testing
  prediccion <- predict(modelo, data[-train_rows], type = "prob")

  prob_baja2 <- prediccion[, "BAJA+2"]
  ganancia_testing <- data[-train_rows, sum((prob_baja2 > 0.025) * ifelse(clase_ternaria == "BAJA+2", 117000, -3000))]

  # normalizo la ganancia
  ganancia_testing_normalizada <- ganancia_testing / 0.3

  return(ganancia_testing_normalizada)
}


In [3]:
ArbolMontecarlo <- function(semillas, data, x, train = 0.70) {
  vector_ganancias <- c() # vector donde voy a ir acumulando las ganancias
  for (semilla in ksemillas)
  {
    ganancia <- GananciaArbol(semilla, dataset, x = x, train = 0.70)
    vector_ganancias <- c(vector_ganancias, ganancia)
  }

  return(mean(vector_ganancias))
}


In [5]:
# dataset <- fread("./datasets/dataset_pequeno.csv") # cargo el dataset #cargo el dataset
dataset <- dataset[foto_mes == 202107] # me quedo solo con el periodo 202107


In [6]:
ksemillas <- c(100297, 422578, 492641, 301039, 723721) # reemplazar por las propias semillas


In [7]:
# defino el primer set de hiperparametros
param1 <- list(
    "cp" = -0.5,
    "minsplit" = 900,
    "minbucket" = 440,
    "maxdepth" = 5
)


llamo a la fucion  ArbolMontecarlo con las 5 semillas de ksemillas

In [13]:
train_p <- 0.7

In [14]:
ganancia_montecarlo1 <- ArbolMontecarlo(ksemillas, dataset, x = param1, train = train_p)


In [15]:
# el SEGUNDO set de hiperparametros
param2 <- list(
    "cp" = -0.5,
    "minsplit" = 1340,
    "minbucket" = 600,
    "maxdepth" = 6
)


In [16]:
ganancia_montecarlo2 <- ArbolMontecarlo(ksemillas, dataset, x = param2, train = 0.70)


In [17]:
ganancia_montecarlo1
ganancia_montecarlo2


[1] 48768000

[1] 48602000

Segun este experimento, el primer juego de hiperparametros es mejor que el primero
<br>
¿Se cumplirá esto subiendo a Kaggle?
<br>
¿Qué sucede si en Kaggle funciona bastante mejor el segundo set de hiperparámetros?

Se cumplirá esto subiendo a Kaggle? No necesariamente.
¿Qué sucede si en Kaggle funciona bastante mejor el segundo set de hiperparámetros? podria revisar como mis hiperparametros se estan comportando con distintos set de entren y prueba.